# BEYlink — Beirut Urban Intelligence

An AI-driven urban analysis tool for Beirut neighbourhoods.

## Datasets
| File | Description |
|------|-------------|
| `karim_beirut.csv` | 7 key intervention plots with images |
| `500_cluster_ID.csv` | 1,964 parcels with POI counts at 500m radius |
| `Hospital.csv` | 30 hospitals scraped from Google Maps |
| `college.csv` | 60 colleges from Google Maps |
| `school.csv` | 80 schools from Google Maps |
| `cafe.csv` | 142 cafes from OpenStreetMap |
| `comunity_center.csv` | 9 community centres from OSM |
| `Parking_data11.csv` | 77 parking nodes from OSM |
| `Parcels_Beirut22.csv` | 21,144 parcel geometries |

## Pipeline
1. Install dependencies and import libraries
2. Load all CSVs and convert coordinates
3. Parse Google Maps URLs to extract hospital/college/school locations
4. Generate POI radar charts per plot
5. Build an interactive multi-layer Folium map
6. Launch Gradio web-app interface

In [ ]:
# Install required packages (safe to re-run)
!pip install -q pyproj folium gradio pandas matplotlib requests pillow

In [ ]:
import re, csv, io, base64, json, math

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pyproj
import folium
from folium.plugins import HeatMap, MarkerCluster

In [ ]:
# ── Load data — works in Colab (clones GitHub repo) and locally ──────────
import os, sys

try:
    import google.colab  # noqa: F401  — only importable inside Colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    REPO_URL = "https://github.com/KarimAbillama/BEYlink.git"
    REPO_DIR = "/content/BEYlink"
    if not os.path.isdir(REPO_DIR):
        os.system(f"git clone {REPO_URL} {REPO_DIR}")
    DATA_DIR = REPO_DIR + "/"
else:
    DATA_DIR = ""  # running locally — CSVs in the same folder

# Core intervention plots (7 rows)
poi_data = pd.read_csv(DATA_DIR + "karim_beirut.csv")

# Full parcel-level cluster data (1,964 rows)
cluster_500 = pd.read_csv(DATA_DIR + "500_cluster_ID.csv")

print(f"Data directory: {DATA_DIR!r}")
print(f"Core plots: {len(poi_data)} rows")
print(f"Cluster parcels: {len(cluster_500)} rows")

In [ ]:
# ── Convert 500-cluster projected coordinates to lat/lng ──────────────────
# The cluster CSVs use an offset UTM Zone 36N coordinate system:
#   easting  = Y_column - 3,280,000
#   northing = X_column - 200,000
transformer = pyproj.Transformer.from_crs("EPSG:32636", "EPSG:4326", always_xy=True)

eastings  = cluster_500["Y"].values - 3_280_000
northings = cluster_500["X"].values - 200_000
lngs, lats = transformer.transform(eastings, northings)
cluster_500["lat"] = lats
cluster_500["lng"] = lngs

# poi_data already uses decimal degrees: X = longitude, Y = latitude
poi_data["lat"] = poi_data["Y"]
poi_data["lng"] = poi_data["X"]

print(f"cluster_500 lat [{lats.min():.4f}, {lats.max():.4f}]  lng [{lngs.min():.4f}, {lngs.max():.4f}]")
print(f"poi_data    lat [{poi_data['lat'].min():.4f}, {poi_data['lat'].max():.4f}]  lng [{poi_data['lng'].min():.4f}, {poi_data['lng'].max():.4f}]")
print("✓ Both datasets confirmed in Beirut area")

In [ ]:
def parse_gmaps_csv(filepath):
    """Extract name, lat, lng from Google Maps scraped CSVs."""
    results = []
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # header
        next(reader, None)  # skip empty row
        for row in reader:
            if not row or not row[0].startswith('http'):
                continue
            m = re.search(r'!3d([\d.]+)!4d([\d.]+)', row[0])
            if m:
                results.append({
                    'name': row[1] if len(row) > 1 else '',
                    'lat': float(m.group(1)),
                    'lng': float(m.group(2)),
                    'rating': row[2] if len(row) > 2 else ''
                })
    return results

hospitals = parse_gmaps_csv(DATA_DIR + 'Hospital.csv')
colleges = parse_gmaps_csv(DATA_DIR + 'college.csv')
schools = parse_gmaps_csv(DATA_DIR + 'school.csv')

print(f'Parsed: {len(hospitals)} hospitals, {len(colleges)} colleges, {len(schools)} schools')

In [ ]:
POI_COLUMNS = [
    'Resort', 'Playground', 'NGO', 'Government', 'Diplomatic', 'Coworking',
    'Company', 'Viewpoint', 'Artwork', 'Restaurant', 'Pub', 'Hotel',
    'Guest House', 'Cafe', 'Worship', 'Museum', 'Theatre', 'Library',
    'Hospital', 'University', 'School', 'Fitness Centre'
]

CLUSTER_LABELS = {
    9: 'Leisure hub', 8: 'Artistic and Touristic hub',
    5: 'Civic artist hub', 4: 'Active hub',
    3: 'Unity district', 1: 'Recreation hub'
}

CLUSTER_COLORS = {
    1: 'green', 3: 'orange', 4: 'red',
    5: 'purple', 8: 'pink', 9: 'darkgreen'
}

CLUSTER_POI_COLS = [
    'Soccer','Basketball','Tennis','Ruins','ArtInstall','NGO','Government',
    'Diplomatic','Coworking','Company','ArtGallery','Restaurant','Hotel',
    'Cafe','Worship','Museum','Theatre','Hospital','University','School'
]

poi_data['clusterID'] = poi_data['clusterID'].replace(CLUSTER_LABELS)
cluster_500['poi_total'] = cluster_500[CLUSTER_POI_COLS].sum(axis=1)

# ── Cluster summary ──────────────────────────────────────────────────────
print('Cluster Distribution:')
for cid, label in sorted(CLUSTER_LABELS.items()):
    subset = cluster_500[cluster_500['clusterID'] == cid]
    if len(subset) > 0:
        print(f'  {label}: {len(subset)} parcels, avg POI = {subset["poi_total"].mean():.1f}')

In [ ]:
INTERVENTIONS = {
    'leisure hub':                ['Public Recreation', 'Pocket Park', 'Food Truck'],
    'artistic and touristic hub': ['Public Park', 'Urban Farming', 'Theatre and Cultural Events',
                                   'Seasonal Market', 'Artisanal Workshops'],
    'civic artist hub':           ['Urban Farming', 'Artisanal Workshops', 'Music Performance',
                                   'Workshop', 'Comfortable Gathering Green Space'],
    'active hub':                 ['Farming', 'Art Exhibition', 'Weekend Market', 'Live Music Performance'],
    'unity district':             ['Urban Garden', 'Urban Farming', 'Entertainment Space', 'Outdoor Gym'],
    'recreation hub':             ['Mobile Food Trucks', 'Pop-up Events', 'Art Installation']
}

BENEFITS = {
    'public recreation':               'Brings people together, improves health, boosts local economy.',
    'pocket park':                     'Green spaces for relaxation, community, and well-being.',
    'food truck':                      'Convenient dining, supports local businesses, adds vibrancy.',
    'public park':                     'Community gatherings, well-being, safety, property values.',
    'urban farming':                   'Fresh produce, sustainability, food security, community engagement.',
    'theatre and cultural events':     'Community engagement, diversity, local talent, economy.',
    'seasonal market':                 'Fresh products, local businesses, community, economy.',
    'artisanal workshops':             'Creativity, skill development, cultural preservation.',
    'music performance':               'Joy, togetherness, local talent, economy.',
    'workshop':                        'Learning, skill development, community, creativity.',
    'comfortable gathering green space':'Inviting areas for gathering and socialising.',
    'farming':                         'Fresh produce, sustainability, food security, education.',
    'art exhibition':                  'Cultural enrichment, community, local artists, creativity.',
    'weekend market':                  'Local products, businesses, community, economy.',
    'live music performance':          'Joy, togetherness, local talent, economy.',
    'urban garden':                    'Fresh produce, sustainability, community, beauty.',
    'entertainment space':             'Events, social interactions, belonging, economy.',
    'outdoor gym':                     'Free exercise, fitness, well-being, outdoor recreation.',
    'mobile food trucks':              'Diverse dining, local businesses, vibrancy.',
    'pop-up events':                   'Unique experiences, community, economy.',
    'art installation':                'Beauty, creativity, community engagement, aesthetics.'
}

In [ ]:
for _, row in poi_data.iterrows():
    values = row[POI_COLUMNS].values.astype(float)
    angles = np.linspace(0, 2 * np.pi, len(POI_COLUMNS), endpoint=False)
    angles_c = np.concatenate((angles, [angles[0]]))
    values_c = np.concatenate((values, [values[0]]))

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.plot(angles_c, values_c, linewidth=1, linestyle='solid')
    ax.fill(angles_c, values_c, alpha=0.25)
    ax.set_xticks(angles)
    ax.set_xticklabels(POI_COLUMNS, fontsize=7)
    ax.set_title(f"Plot Area {row['PLOT AREA']}")
    plt.tight_layout()
    plt.show()
    plt.close(fig)

## Interactive Map

The map below includes six toggle-able layers:
1. **POI Density Heatmap** — 1,964 parcels coloured by total POI count
2. **All 1,964 Parcels** — circle markers coloured by cluster type
3. **Intervention Plots** — 7 key sites with images, radar charts, and intervention proposals
4. **Hospitals** — 30 locations with ratings
5. **Schools** — 80 locations
6. **Colleges** — 60 locations

Use the layer control panel (top-right) to toggle each layer on/off.

In [ ]:
beirut_map = folium.Map(location=[33.8938, 35.5018], zoom_start=13, control_scale=True)
folium.TileLayer('cartodbdark_matter', name='Dark').add_to(beirut_map)
folium.TileLayer('OpenStreetMap', name='Street').add_to(beirut_map)
folium.TileLayer('cartodbpositron', name='Light').add_to(beirut_map)

# ── Layer 1: POI density heatmap ────────────────────────────────────────────
heat_data = cluster_500[['lat', 'lng', 'poi_total']].values.tolist()
heat_fg = folium.FeatureGroup(name='POI Density Heatmap (1964 parcels)')
HeatMap(heat_data, radius=15, blur=20, max_zoom=16,
        gradient={0.2:'blue', 0.4:'cyan', 0.6:'lime', 0.8:'yellow', 1.0:'red'}
).add_to(heat_fg)
heat_fg.add_to(beirut_map)

# ── Layer 2: All 1964 parcels (coloured by cluster) ─────────────────────────
cluster_fg = folium.FeatureGroup(name='All 1964 Parcels (by cluster)', show=False)
for _, row in cluster_500.iterrows():
    cid = int(row['clusterID'])
    label = CLUSTER_LABELS.get(cid, f'Cluster {cid}')
    color = CLUSTER_COLORS.get(cid, 'gray')
    folium.CircleMarker(
        [row['lat'], row['lng']], radius=3,
        color=color, fill=True, fill_opacity=0.7,
        popup=f'<b>{label}</b><br>Area: {int(row["area real"])} m²<br>POI Total: {int(row["poi_total"])}'
    ).add_to(cluster_fg)
cluster_fg.add_to(beirut_map)

# ── Layer 3: Core intervention plots (7 sites) ──────────────────────────────
intervention_fg = folium.FeatureGroup(name='Intervention Plots (7 sites)')
for index, loc in poi_data.iterrows():
    cluster_id = str(loc['clusterID']).lower()
    iv_list = INTERVENTIONS.get(cluster_id, [])

    values = loc[POI_COLUMNS].values.astype(float)
    angles = np.linspace(0, 2*np.pi, len(POI_COLUMNS), endpoint=False).tolist()
    fig, ax = plt.subplots(figsize=(4,4), subplot_kw=dict(polar=True))
    ax.plot(angles+angles[:1], np.append(values,values[0]), color='purple', linewidth=2)
    ax.fill(angles+angles[:1], np.append(values,values[0]), color='purple', alpha=0.25)
    ax.set_xticks(angles); ax.set_xticklabels(POI_COLUMNS, fontsize=6)
    ax.set_yticks([1,2,3,4,5])
    ax.set_yticklabels(['1','2','3','4','5'], color='grey', fontsize=8)
    ax.set_ylim(0,5)
    ax.set_title(f'Cluster: {loc["clusterID"]}', size=11, y=1.1)
    ax.grid(True)
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=80, bbox_inches='tight')
    plt.close(fig); buf.seek(0)
    chart_b64 = base64.b64encode(buf.getvalue()).decode()

    local_benefits = {iv: BENEFITS.get(iv.lower(), 'Coming soon.') for iv in iv_list}
    benefits_json = json.dumps(local_benefits)
    opts = '<option value="">Select an intervention</option>'
    for iv in iv_list:
        opts += f'<option value="{iv}">{iv}</option>'

    html = f'''
    <style>body{{font-family:sans-serif;font-size:13px;margin:0}}
    .lbl{{font-weight:bold;margin-top:8px;display:block}}
    select{{width:100%;margin:6px 0;padding:4px}}</style>
    <img src="{loc['Image Url']}" alt="Site" width="340"
         style="border-radius:4px;display:block;margin-bottom:6px"
         onerror="this.style.display='none'">
    <span class="lbl">Cluster: {loc['clusterID']}</span>
    <span class="lbl">Interventions:</span>
    <select id="iv_{index}" onchange="sc_{index}(this)">{opts}</select>
    <div id="c_{index}" style="display:none">
        <span class="lbl">Benefits:</span>
        <p id="bt_{index}"></p>
    </div>
    <img src="data:image/png;base64,{chart_b64}" alt="Radar" width="340">
    <script>(function(){{var b={benefits_json};
    window.sc_{index}=function(s){{var v=s.value,
    c=document.getElementById('c_{index}'),
    t=document.getElementById('bt_{index}');
    if(v){{t.innerText=b[v]||'Coming soon.';c.style.display='block'}}
    else{{c.style.display='none'}}}}}})();</script>
    '''
    iframe = folium.IFrame(html=html, width=380, height=520)
    folium.Marker(
        [loc['lat'], loc['lng']], popup=folium.Popup(iframe, max_width=500),
        icon=folium.Icon(color='purple', icon='star')
    ).add_to(intervention_fg)
intervention_fg.add_to(beirut_map)

# ── Layer 4: Hospitals ───────────────────────────────────────────────────────
hosp_fg = folium.FeatureGroup(name=f'Hospitals ({len(hospitals)})', show=False)
hosp_mc = MarkerCluster().add_to(hosp_fg)
for h in hospitals:
    folium.Marker(
        [h['lat'], h['lng']],
        popup=f'<b>{h["name"]}</b><br>Rating: {h.get("rating","-")}',
        icon=folium.Icon(color='red', icon='plus-sign')
    ).add_to(hosp_mc)
hosp_fg.add_to(beirut_map)

# ── Layer 5: Schools ─────────────────────────────────────────────────────────
school_fg = folium.FeatureGroup(name=f'Schools ({len(schools)})', show=False)
school_mc = MarkerCluster().add_to(school_fg)
for s in schools:
    folium.Marker(
        [s['lat'], s['lng']], popup=f'<b>{s["name"]}</b>',
        icon=folium.Icon(color='blue', icon='book')
    ).add_to(school_mc)
school_fg.add_to(beirut_map)

# ── Layer 6: Colleges ────────────────────────────────────────────────────────
coll_fg = folium.FeatureGroup(name=f'Colleges ({len(colleges)})', show=False)
coll_mc = MarkerCluster().add_to(coll_fg)
for c in colleges:
    folium.Marker(
        [c['lat'], c['lng']], popup=f'<b>{c["name"]}</b>',
        icon=folium.Icon(color='green', icon='education')
    ).add_to(coll_mc)
coll_fg.add_to(beirut_map)

folium.LayerControl(collapsed=False).add_to(beirut_map)

# Auto-fit map to show all data
all_lats = list(cluster_500['lat']) + list(poi_data['lat'])
all_lngs = list(cluster_500['lng']) + list(poi_data['lng'])
beirut_map.fit_bounds([[min(all_lats), min(all_lngs)], [max(all_lats), max(all_lngs)]])

beirut_map

## Gradio App

Interactive web interface to explore individual plots on the map.

In [ ]:
import gradio as gr
from folium.raster_layers import ImageOverlay

def visualize_map(latitude, longitude, plot_image_url, plot_description):
    m = folium.Map(location=[latitude, longitude], zoom_start=14)
    folium.Marker(
        [latitude, longitude], popup='Selected Plot',
        icon=folium.Icon(color='purple')
    ).add_to(m)

    if plot_image_url and plot_image_url.strip().startswith('http'):
        bounds = [
            [latitude - 0.005, longitude - 0.005],
            [latitude + 0.005, longitude + 0.005]
        ]
        ImageOverlay(plot_image_url, bounds=bounds, opacity=0.6).add_to(m)

    map_html = m.get_root().render()
    desc_html = (
        f'<div style="font-family:sans-serif;padding:16px">'        f'<h3>{plot_description}</h3></div>'
        if plot_description else '<p style="color:#888">No description provided.</p>'
    )
    return map_html, desc_html

with gr.Blocks(title='BEYlink — Plot Visualiser') as demo:
    gr.Markdown('## BEYlink — Plot Visualiser
Visualise a Beirut plot on an interactive map.')

    with gr.Row():
        with gr.Column(scale=1):
            lat_in  = gr.Number(label='Latitude',  value=33.8938)
            lng_in  = gr.Number(label='Longitude', value=35.5018)
            url_in  = gr.Textbox(label='Plot Image URL', placeholder='https://...')
            desc_in = gr.Textbox(label='Plot Description', lines=4,
                                 placeholder='Describe this plot...')
            btn = gr.Button('Show on Map', variant='primary')

    with gr.Row():
        map_out  = gr.HTML(label='Map',         elem_id='map-panel')
        desc_out = gr.HTML(label='Description', elem_id='desc-panel')

    btn.click(
        fn=visualize_map,
        inputs=[lat_in, lng_in, url_in, desc_in],
        outputs=[map_out, desc_out]
    )

    demo.css = '#map-panel { flex: 3 !important; } #desc-panel { flex: 1 !important; }'

demo.launch(share=True)